## Laser Heating Inclusion in Plasma Decay

In [2]:
import sys
import os
dir = os.path.dirname(r".\PLPAK")
sys.path.append(dir)


import PLPAK as pl
print("PLPAK version: ", pl.__version__)

PLPAK version:  0.1.0


## Add the laser heating model to existing plasmaSystems class.

Here, use the same plasmaSystems class and based on it extend the dTdt_ method to include laser heating. The new model will be derived from the existing model of N2plasma.

In [ ]:
# From the library, use one of the plasmaSystems to create a new derived class

# The class to base the new class on
baseClass = pl.plasmaSystems.N2Plasma_54Rxn # can change this to any of the plasmaSystems when needed

# make the new class, only need to change the dTdt_ method.
class N2Plasma_54Rxn_Heating(baseClass):

    # instantiate the class
    def __init__(self, Ysp, T, verbose = False, **kwargs):
        # new variables from **kwargs
        